# **Índice de habitabilidad basado en las métricas obtenidas y clasificar las ciudades en categorías**

In [ ]:
import pandas as pd  # Importar la librería pandas para manejar datos en formato de tablas (dataframes)

# Agregar una nueva columna al DataFrame para calcular el índice de habitabilidad
def calcular_indice_habitabilidad(df):
    # Normalizar las métricas para que todas tengan valores comparables

    # Calcular un índice para la calidad del aire basado en el AQI (Air Quality Index). Invertimos el AQI para que valores más bajos (mejor calidad) tengan un puntaje más alto.
    df['indice_calidad_aire'] = 100 - df['aqi_us']  # Invertir AQI, valores más bajos son mejores

    # Calcular el índice de temperatura: Si está en el rango de 15-30°C, se considera ideal, de lo contrario se asigna un valor de 50.
    df['indice_temperatura'] = df['temperatura_c_x'].apply(lambda x: 100 if 15 <= x <= 30 else 50)

    # Calcular el índice de humedad: Si está en el rango de 30%-60%, se considera ideal, de lo contrario se asigna un valor de 50.
    df['indice_humedad'] = df['humedad_x'].apply(lambda x: 100 if 30 <= x <= 60 else 50)

    # Calcular el índice de viento: Si la velocidad del viento es mayor o igual a 2 m/s, se considera ideal, de lo contrario se asigna un valor de 50.
    df['indice_viento'] = df['velocidad_viento_m/s'].apply(lambda x: 100 if x >= 2 else 50)

    # Promediar los índices para obtener un índice general de habitabilidad
    # El índice de habitabilidad es el promedio de los índices de calidad del aire, temperatura, humedad y viento.
    df['indice_habitabilidad'] = (
        df['indice_calidad_aire'] +  # Índice de calidad del aire
        df['indice_temperatura'] +  # Índice de temperatura
        df['indice_humedad'] +      # Índice de humedad
        df['indice_viento']         # Índice de viento
    ) / 4  # Promedio de los 4 índices

    # Clasificar según umbrales
    # Función que clasifica la habitabilidad en categorías de "Alta Habitabilidad", "Moderada Habitabilidad" y "Baja Habitabilidad"
    def clasificar_habitabilidad(indice):
        if indice >= 75:  # Si el índice es mayor o igual a 75, la ciudad tiene alta habitabilidad
            return "Alta Habitabilidad"
        elif indice >= 50:  # Si el índice es mayor o igual a 50, pero menor a 75, la ciudad tiene moderada habitabilidad
            return "Moderada Habitabilidad"
        else:  # Si el índice es menor a 50, la ciudad tiene baja habitabilidad
            return "Baja Habitabilidad"

    # Aplicamos la función de clasificación a la columna 'indice_habitabilidad' para crear una nueva columna 'categoria_habitabilidad'
    df['categoria_habitabilidad'] = df['indice_habitabilidad'].apply(clasificar_habitabilidad)

    return df  # Devolvemos el DataFrame con las nuevas columnas y categorías

# Aplicar la función al DataFrame de comparación
comparacion = calcular_indice_habitabilidad(comparacion)  # Calculamos el índice de habitabilidad para cada ciudad

# Mostrar los resultados con las columnas relevantes
print("\nClasificación de habitabilidad de las ciudades:")
# Mostramos la ciudad, el índice de habitabilidad y la categoría de habitabilidad
print(comparacion[['ciudad', 'indice_habitabilidad', 'categoria_habitabilidad']])

# Identificar la ciudad con el índice de habitabilidad más alto (la mejor ciudad)
ciudad_ideal = comparacion.loc[comparacion['indice_habitabilidad'].idxmax()]  # Encontramos la ciudad con el índice máximo
print("\nLa ciudad con mayor habitabilidad es:")  # Mostramos el resultado
print(f"Ciudad: {ciudad_ideal['ciudad']}")
print(f"Índice de Habitabilidad: {ciudad_ideal['indice_habitabilidad']:.2f}")  # Mostramos el índice con dos decimales
print(f"Categoría: {ciudad_ideal['categoria_habitabilidad']}")  # Mostramos la categoría de la ciudad ideal

## **Evaluación de Riesgos para la salud**

In [ ]:
import pandas as pd  # Importa la biblioteca pandas para manipular y analizar datos tabulares

# Función para evaluar riesgos en base al AQI y condiciones climáticas extremas
def evaluar_riesgos_salud(df):
    """
    Esta función evalúa los riesgos para la salud de cada ciudad en función de la calidad del aire (AQI),
    la temperatura y la humedad, clasificando el riesgo como alto, moderado o bajo.
    """

    # Crear una nueva columna para evaluar el riesgo de calidad del aire según el AQI
    def riesgo_calidad_aire(aqi):
        """
        Evalúa el riesgo relacionado con la calidad del aire basándose en el AQI.
        El AQI más alto indica un mayor riesgo para la salud.
        """
        if aqi > 150:
            return "Riesgo alto de problemas respiratorios (población general)"
        elif aqi > 100:
            return "Riesgo moderado para personas sensibles"
        else:
            return "Riesgo bajo"

    # Crear una nueva columna para evaluar el riesgo por temperatura extrema
    def riesgo_temperatura(temperatura):
        """
        Evalúa el riesgo relacionado con temperaturas extremas.
        Las temperaturas altas o bajas extremas representan riesgos para la salud.
        """
        if temperatura > 35:
            return "Riesgo alto debido a calor extremo"
        elif temperatura < 0:
            return "Riesgo alto debido a frío extremo"
        else:
            return "Riesgo bajo"

    # Crear una nueva columna para evaluar el riesgo por humedad extrema
    def riesgo_humedad(humedad):
        """
        Evalúa el riesgo relacionado con niveles de humedad extremas.
        La alta o baja humedad puede afectar la respiración y la salud en general.
        """
        if humedad > 70:
            return "Riesgo alto por humedad excesiva"
        elif humedad < 20:
            return "Riesgo alto por baja humedad"
        else:
            return "Riesgo bajo"

    # Evaluar los riesgos para cada ciudad, asignando valores a las columnas
    df['riesgo_aire'] = df['aqi_us'].apply(riesgo_calidad_aire)  # Se aplica la función de riesgo por aire al AQI de cada ciudad
    df['riesgo_temperatura'] = df['temperatura_c_x'].apply(riesgo_temperatura)  # Se aplica la función de riesgo por temperatura a cada ciudad
    df['riesgo_humedad'] = df['humedad_x'].apply(riesgo_humedad)  # Se aplica la función de riesgo por humedad a cada ciudad

    # Función para calcular el riesgo global de la salud combinando los riesgos de aire, temperatura y humedad
    def riesgo_global(row):
        """
        Combina los riesgos por calidad del aire, temperatura y humedad para determinar el riesgo global.
        Si cualquiera de los riesgos es alto, el riesgo global será alto.
        """
        riesgos = [row['riesgo_aire'], row['riesgo_temperatura'], row['riesgo_humedad']]  # Lista de todos los riesgos de la ciudad
        # Si cualquiera de los riesgos es alto, el riesgo global es alto. Si es moderado, el riesgo global es moderado.
        if "Riesgo alto" in riesgos:
            return "Riesgo alto para la salud"
        elif "Riesgo moderado" in riesgos:
            return "Riesgo moderado para la salud"
        else:
            return "Riesgo bajo para la salud"

    # Crear una columna para el riesgo global
    df['riesgo_global'] = df.apply(riesgo_global, axis=1)  # Aplica la función de riesgo global a cada fila del DataFrame

    return df  # Devuelve el DataFrame con las nuevas columnas de riesgos

# Aplicar la función a los datos obtenidos
comparacion = evaluar_riesgos_salud(comparacion)  # Llama a la función y le pasa el DataFrame 'comparacion' con datos de ciudades

# Mostrar los resultados con las evaluaciones de riesgos
print("\nEvaluación de riesgos para la salud en las ciudades:")
# Muestra una tabla con la ciudad y los distintos riesgos
print(comparacion[['ciudad', 'riesgo_aire', 'riesgo_temperatura', 'riesgo_humedad', 'riesgo_global']])

# Identificar la ciudad con el menor riesgo para la salud
ciudad_sin_riesgo = comparacion.loc[comparacion['riesgo_global'] == "Riesgo bajo para la salud"]  # Filtra las ciudades con riesgo bajo
print("\nCiudades con el menor riesgo para la salud:")
# Itera a través de las ciudades con el menor riesgo y muestra los resultados
for index, row in ciudad_sin_riesgo.iterrows():
    print(f"Ciudad: {row['ciudad']}")  # Muestra el nombre de la ciudad
    print(f"Riesgo de aire: {row['riesgo_aire']}")  # Muestra el riesgo asociado al aire
    print(f"Riesgo de temperatura: {row['riesgo_temperatura']}")  # Muestra el riesgo asociado a la temperatura
    print(f"Riesgo de humedad: {row['riesgo_humedad']}")  # Muestra el riesgo asociado a la humedad
    print(f"Riesgo global: {row['riesgo_global']}")  # Muestra el riesgo global para la salud de la ciudad


Este enfoque permite evaluar los riesgos para la salud respiratoria en diversas ciudades del Perú en función de la calidad del aire y las condiciones climáticas extremas.